In [127]:
import pandas as pd
from kmodes.kmodes import KModes
from kmodes.kprototypes import KPrototypes
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

df_profile = pd.read_excel("A2 - Profile.xlsx")


In [128]:
#preprocessing
df_profile = df_profile[["Age", "Job", "Marital_status", "Education", "Credict_Payment"]]
df_profile = df_profile.dropna(subset=["Age", "Job", "Marital_status", "Education", "Credict_Payment"])
df_profile = df_profile.drop(df_profile[(df_profile["Job"] == "unknown")|(df_profile["Marital_status"] == "unknown")|(df_profile["Education"] == "unknown")|(df_profile["Credict_Payment"] == "unknown")].index)

In [129]:
to_encode = ["Job", "Marital_status", "Education", "Credict_Payment"]
d={}
for col in to_encode:
    d[col]= LabelEncoder().fit(df_profile[col])
    df_profile[col] = d[col].transform(df_profile[col])

df_profile

,Age,Job,Marital_status,Education,Credict_Payment
0,30,1,1,2,0
1,39,7,2,3,0
2,25,7,1,3,0
4,47,0,1,5,0
5,32,7,2,5,0
...,...,...,...,...,...
1808,34,7,1,3,0
1809,49,4,1,5,0
1810,50,1,0,0,0
1811,51,2,1,5,0


In [130]:
scaler = StandardScaler()
df_profile["Age"] = scaler.fit_transform(df_profile[["Age"]])
df_profile

,Age,Job,Marital_status,Education,Credict_Payment
0,-0.941670,1,1,2,0
1,-0.065910,7,2,3,0
2,-1.428203,7,1,3,0
4,0.712543,0,1,5,0
5,-0.747057,7,2,5,0
...,...,...,...,...,...
1808,-0.552443,7,1,3,0
1809,0.907157,4,1,5,0
1810,1.004463,1,0,0,0
1811,1.101770,2,1,5,0


In [131]:
# fit
km = KPrototypes(n_clusters=5, init='Cao', verbose=1)
clusters = km.fit_predict(df_profile, categorical= [1,2,3,4])

Initialization method and algorithm are deterministic. Setting n_init to 1.
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 292, ncost: 1820.2148172851166
Run: 1, iteration: 2/100, moves: 104, ncost: 1809.991196740859
Run: 1, iteration: 3/100, moves: 202, ncost: 1740.154322589214
Run: 1, iteration: 4/100, moves: 256, ncost: 1647.9239922274537
Run: 1, iteration: 5/100, moves: 269, ncost: 1626.992356199789
Run: 1, iteration: 6/100, moves: 76, ncost: 1623.6338474083434
Run: 1, iteration: 7/100, moves: 39, ncost: 1622.1112086340386
Run: 1, iteration: 8/100, moves: 6, ncost: 1622.056518199609
Run: 1, iteration: 9/100, moves: 0, ncost: 1622.056518199609
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 347, ncost: 1719.2624227821857
Run: 2, iteration: 2/100, moves: 143, ncost: 1678.5194361988522
Run: 2, iteration: 3/100, moves: 163, ncost: 1657.1180870379494
Run: 2,

In [132]:
#inverse numerical scaling and encoding
for col in to_encode:
    df_profile[col] = d[col].inverse_transform(df_profile[col])

age_inverse = pd.DataFrame(df_profile, columns=["Age"])
age_inverse[["Age"]] = scaler.inverse_transform(age_inverse[["Age"]])


In [133]:
# Add the cluster labels to the original DataFrame
df_profile['cluster'] = clusters
df_profile["Age"] = age_inverse["Age"]
print(df_profile.head())

    Age          Job Marital_status          Education Credict_Payment  \
0  30.0  blue-collar        married           basic.9y              no   
1  39.0     services         single        high.school              no   
2  25.0     services        married        high.school              no   
4  47.0       admin.        married  university.degree              no   
5  32.0     services         single  university.degree              no   

   cluster  
0        4  
1        3  
2        4  
4        0  
5        2  


In [134]:
df_profile.to_csv("KP_clusters.csv")